In [1]:
import tensorflow as tf
from tensorflow.python.tools import inspect_checkpoint as chkp
import os

In [2]:
save_dir = 'checkpoints'

# Save

In [3]:
a = tf.Variable(1.5, name='a')
b = tf.Variable(2.5, name='b')

In [4]:
tf.global_variables()

[<tf.Variable 'a:0' shape=() dtype=float32_ref>,
 <tf.Variable 'b:0' shape=() dtype=float32_ref>]

* .meta file: save meta graph, a protocol buffer which saves a tensorflow graph
* .data-?????-of-????? and .index file: save the values of the variables

In [5]:
saver = tf.train.Saver()
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    print(a.eval(sess))
    print(b.eval(sess))
    saver.save(sess, os.path.join(save_dir, 'model.ckpt'), 
               write_meta_graph=True # also generate .meta file
              )

1.5
2.5


# Restore

In [6]:
tf.reset_default_graph()

In [7]:
tf.global_variables()

[]

In [8]:
v1 = tf.get_variable(name='a', dtype=tf.float32, shape=())
v2 = tf.get_variable(name='b', dtype=tf.float32, shape=())

W0730 23:56:54.507832  7652 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, os.path.join(save_dir, 'model.ckpt'))
    
    print(v1.eval(sess))
    print(v2.eval(sess))

W0730 23:56:54.667406  7652 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


1.5
2.5


# Inspect checkpoint

In [10]:
chkp.print_tensors_in_checkpoint_file(file_name=os.path.join(save_dir, 'model.ckpt'), tensor_name='', all_tensors=True)

tensor_name:  a
1.5
tensor_name:  b
2.5
# Total number of params: 2


In [11]:
chkp.print_tensors_in_checkpoint_file(file_name=os.path.join(save_dir, 'model.ckpt'), tensor_name='b', all_tensors=False)

tensor_name:  b
2.5
# Total number of params: 2


# Import meta graph (restore model)

In [12]:
tf.reset_default_graph()

In [13]:
tf.global_variables()

[]

In [14]:
saver = tf.train.import_meta_graph(os.path.join(save_dir, 'model.ckpt.meta'))

In [15]:
tf.global_variables()

[<tf.Variable 'a:0' shape=() dtype=float32_ref>,
 <tf.Variable 'b:0' shape=() dtype=float32_ref>]

In [16]:
with tf.Session() as sess:
    saver.restore(sess, os.path.join(save_dir, 'model.ckpt'))
    print(sess.run('a:0'))
    print(sess.run('b:0'))

1.5
2.5
